#  IMPORTING LIBRARIES 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

# LOADING WEB PAGES WITH 'REQUEST'

In [2]:
url = "https://www.w3schools.com/html/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [3]:
#printing the staus of the requests
print(status)

200


So it's a successful request

# EXTRACTING TITLE WITH BEAUTIFULSOUP

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
# Extracting title of page
page_title = soup.title.text

# prining the result
print(page_title)

HTML Tutorial


# SOUP-ED BODY AND HEAD 

In [6]:
# Extracting body of page
page_body = soup.body

# Extracting head of page
page_head = soup.head

# printing the result
print(page_head)

<head>
<title>HTML Tutorial</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="HTML, Python, CSS, SQL, JavaScript, How to, PHP, Java, C++, jQuery, Bootstrap, C#, Colors, W3.CSS, XML, MySQL, Icons, NodeJS, React, Graphics, Angular, R, AI, Git, Data Science, Code Game, Tutorials, Programming, Web Development, Training, Learning, Quiz, Courses, Lessons, References, Examples, Source code, Demos, Tips" name="Keywords"/>
<meta content="Well organized and easy to understand Web building tutorials with lots of examples of how to use HTML, CSS, JavaScript, SQL, Python, PHP, Bootstrap, Java, XML and more." name="Description"/>
<meta content="https://www.w3schools.com/images/w3schools_logo_436_2.png" property="og:image"/>
<meta content="image/png" property="og:image:type"/>
<meta content="436" property="og:image:width"/>
<meta content="228" property="og:image:height"/>
<meta content="W3Schools offers free online tutorials, referenc

# SELECT WITH BEAUTIFULSOUP

In [7]:
# Extracting first <h1>(...)</h1> text
first_h1 = soup.select('h1')[0].text 
print(first_h1)

HTML Tutorial


# PASSING REQUIRMENTS 

In [8]:
# Creating all_h1_tags as empty list
all_h1_tags = []

# Set all_h1_tags to all h1 tags of the soup
for element in soup.select('h1'):
    all_h1_tags.append(element.text)

# Create seventh_p_text and set it to 7th p element text of the page
seventh_p_text = soup.select('p')[7].text

print(all_h1_tags, seventh_p_text)

['HTML Tutorial'] This HTML tutorial also contains nearly 100 HTML exercises.


# MAIN HEADINGS OF HOME PAGE 

In [9]:
# creating a list of all common heading tags
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> HTML Tutorial
h2 -> HTML Forms
h2 -> HTML Graphics
h2 -> HTML Media
h2 -> HTML APIs
h2 -> HTML Examples
h2 -> HTML References
h1 -> HTML Tutorial
h2 -> Easy Learning with HTML "Try it Yourself"
h3 -> Example
h2 -> HTML Examples
h2 -> HTML Exercises
h2 -> Test Yourself With Exercises
h2 -> Exercise:
h2 -> HTML Quiz Test
h2 -> HTML References
h2 -> Kickstart your career
h2 -> Report Error
h2 -> Thank You For Helping Us!


# SUB HEADINGS OF HTML 

In [10]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left"><span class="left_h2">HTML</span> Tutorial</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Forms</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Graphics</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Media</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> APIs</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Examples</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> References</h2>]

In [11]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

HTML Tutorial
HTML Forms
HTML Graphics
HTML Media
HTML APIs
HTML Examples
HTML References


In [12]:
# contents in html
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

# Lets scrape the data for the column 'HEADINGS', 'SUB-HEADINGS, 'LINKS', and 'DESCRIPTION'

----

In [183]:
# getting the Headings, Sub-Headings, Links and Contents
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []

base_url = "https://www.w3schools.com/html/"
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
            if extra[0] == '/':
                base_url = "https://www.w3schools.com"
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            
            #################################################
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            table_new.append(row)
            page_table.append(table_new)
            ################################################
            
            # Points and Example Code
            # Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                s = s[:-3]
                unorder.append(start_string)
            unordered_lists.append(unorder)
            
            #################################################
        #     print(page_content)
            for page_data in page_content: 
        #         print(page_data)
                page_paragraphs = page_data.find_all('p')
        #         print(page_paragraphs)
            content_per_page = []
            for i in page_paragraphs:
                content_per_page.append(i.text)
            sub_page_content_list.append(content_per_page)
            
            
            
            #time.sleep(5)

In [184]:
length = len(course_links)

In [185]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(page_table), len(unordered_lists)

(77, 77, 77, 77, 77, 77)

In [219]:
print("Heading ---->>>> Sub-Heading ---->>>> Course link ---->>>> Description ----> Unordered List---->>>> Tabular Data")
print('-----------------------------------------------------------------------------------------------------------------------')
for i in range(length):
    try:
        print(f'{i+1}.  {course_heading[i]}--->>> {course_subheading[i]} --->>> {course_links[i]} --->>> {sub_page_content_list[i][0][:5]}... --->>> {unordered_lists[i][0][:5]}--->>> {page_table[i][0][:5]}')
        print('\n')
    except:
        print(f'{i+1}.  {course_heading[i]}--->>> {course_subheading[i]} --->>> {course_links[i]} --->>> {sub_page_content_list[i][0][:5]} --->>> {unordered_lists[i][:5]} --->>> {page_table[i][0][:5]}')
        #print('\n')

Heading ---->>>> Sub-Heading ---->>>> Course link ---->>>> Description ----> Unordered List---->>>> Tabular Data
-----------------------------------------------------------------------------------------------------------------------
1.  HTML Tutorial--->>> HTML HOME --->>> https://www.w3schools.com/html/default.asp --->>> HTML  --->>> [] --->>> 
2.  HTML Tutorial--->>> HTML Introduction --->>> https://www.w3schools.com/html/html_intro.asp --->>> HTML ... --->>> HTML --->>> <h1>|


3.  HTML Tutorial--->>> HTML Editors --->>> https://www.w3schools.com/html/html_editors.asp --->>> A sim --->>> [] --->>> 
4.  HTML Tutorial--->>> HTML Basic --->>> https://www.w3schools.com/html/html_basic.asp --->>> In th --->>> [] --->>> 
5.  HTML Tutorial--->>> HTML Elements --->>> https://www.w3schools.com/html/html_elements.asp --->>> An HT --->>> [] --->>> <h1>|
6.  HTML Tutorial--->>> HTML Attributes --->>> https://www.w3schools.com/html/html_attributes.asp --->>> HTML ... --->>> All H--->>> 


7.  HT

IndexError: list index out of range

# Creating & Saving the dataset into csv file

In [214]:
from csv import writer

with open('html_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description', 'Table Data']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], page_table[i]]
        the_writer.writerow(row)

In [215]:
file = pd.read_csv('html_w3school.csv')
dataset = pd.DataFrame(file)

In [216]:
dataset.head(20)

,Heading,Sub-Heading,Links,Description,Table Data
0,HTML Tutorial,HTML HOME,https://www.w3schools.com/html/default.asp,['HTML is the standard markup language for Web...,['']
1,HTML Tutorial,HTML Introduction,https://www.w3schools.com/html/html_intro.asp,['HTML is the standard markup language for cre...,['<h1>|||My First Heading|||</h1>&&&<p>|||My f...
2,HTML Tutorial,HTML Editors,https://www.w3schools.com/html/html_editors.asp,['A simple text editor is all you need to lear...,['']
3,HTML Tutorial,HTML Basic,https://www.w3schools.com/html/html_basic.asp,['In this chapter we will show some basic HTML...,['']
4,HTML Tutorial,HTML Elements,https://www.w3schools.com/html/html_elements.asp,"['An HTML element is defined by a start tag, s...",['<h1>|||My First Heading|||</h1>&&&<p>|||My f...
5,HTML Tutorial,HTML Attributes,https://www.w3schools.com/html/html_attributes...,['HTML attributes provide additional informati...,['']
6,HTML Tutorial,HTML Headings,https://www.w3schools.com/html/html_headings.asp,['HTML headings are titles or subtitles that y...,"[""<html>|||Defines the root of an HTML documen..."
7,HTML Tutorial,HTML Paragraphs,https://www.w3schools.com/html/html_paragraphs...,"['A paragraph always starts on a new line, and...",['<p>|||Defines a paragraph&&&<hr>|||Defines a...
8,HTML Tutorial,HTML Styles,https://www.w3schools.com/html/html_styles.asp,['The HTML style attribute is used to add styl...,['']
9,HTML Tutorial,HTML Formatting,https://www.w3schools.com/html/html_formatting...,['HTML contains several elements for defining ...,['<b>|||Defines bold text&&&<em>|||Defines emp...


In [217]:
dataset['Table Data'][62]

'[\'accept|||<input>|||Specifies the types of files that the server accepts (only for type="file")&&&accept-charset|||<form>|||Specifies the character encodings that are to be used for the form \\nsubmission&&&accesskey|||Global Attributes|||Specifies a shortcut key to activate/focus an element&&&action|||<form>|||Specifies where to send the form-data when a form is submitted&&&align|||Not supported in HTML 5.|||Specifies the alignment according to surrounding elements. Use CSS instead&&&alt|||<area>, <img>, <input>|||Specifies an alternate text when the original element fails to display&&&async|||<script>|||Specifies that the script is executed asynchronously (only for external \\nscripts)&&&autocomplete|||<form>, <input>|||Specifies whether the <form> or the <input> element should have autocomplete \\nenabled&&&autofocus|||<button>, <input>, <select>, <textarea>|||Specifies that the element should automatically get focus when the page \\nloads&&&autoplay|||<audio>, <video>|||Specifie

In [177]:
u = 'https://www.w3schools.com/html/html_links.asp'
r = requests.get(u)

In [178]:
sp = BeautifulSoup(r.content, 'html.parser')

In [179]:
ul = sp.find_all('ul')

In [166]:
list_types = ['ul', 'ol']

In [167]:
for i in main_body:
    ul = i.find_all('ul')
    

In [180]:
uu = []
for i in ul:
    s = ''
    for j in i:
        if j.text != '\n':
            print(j.text.replace('\n',''))
            s += j.text.replace('\n','') + '@@@'
    s = s[:-3]
    uu.append(s)

An unvisited link is underlined and blue
A visited link is underlined and purple
An active link is underlined and red
_self - Default. Opens the document in   the same window/tab as it was clicked
_blank - Opens the document in a new window or tab
_parent - Opens the document in the parent frame
_top - Opens the document in the full body of the window
Use the <a> element to define a link
Use the href attribute to define the link address
Use the target attribute to define where to open the linked document
Use the <img> element (inside <a>)   to use an image as a link
Use the   mailto: scheme inside the href attribute to create a link that opens the user's email program


In [181]:
uu

['An unvisited link is underlined and blue@@@A visited link is underlined and purple@@@An active link is underlined and red',
 '_self - Default. Opens the document in   the same window/tab as it was clicked@@@_blank - Opens the document in a new window or tab@@@_parent - Opens the document in the parent frame@@@_top - Opens the document in the full body of the window',
 "Use the <a> element to define a link@@@Use the href attribute to define the link address@@@Use the target attribute to define where to open the linked document@@@Use the <img> element (inside <a>)   to use an image as a link@@@Use the   mailto: scheme inside the href attribute to create a link that opens the user's email program"]

In [173]:
dl = sp.find_all('dl')

In [174]:
dl

[]